In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


In [3]:
#Create New Target (Good in 60-15 but bad in 90-24)
model_data['G15_B24'] = model_data.apply(
   lambda row: 0 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 0)
    else ( 1 if (row['60_in_15mob'] == 0 and row['90_in_24mob'] == 1)
    else 2),axis=1)

model_data= model_data[model_data['G15_B24']!=2]
print('********* New Target Prepared *********')

********* New Target Prepared *********


In [4]:
data = model_data.copy()
# take one copy for model development data
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [7]:
#Split the time interval for model building and Validation of oot
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2022-06')]
oot_df1= data[(data['month_year']>='2022-07') & (data['month_year']<='2022-09')]
print('********* Data Splitted *********')
print(model_df1.shape, oot_df1.shape)

********* Data Splitted *********
(34344, 2832) (10802, 2832)


In [8]:
features_list = [
'num_PERSONAL LOAN_enq_last_24M',
'num_CONSUMER LOAN_enq_last_9M',
'total_emi',
'num_of_loans_le_1lac_non_credit',
'num_CREDIT CARD_enq_last_3M',
'Utilization_all',
'Total_outstanding_Balance_CREDIT CARD_opened_last_6M',
'Age_of_oldest_open_account',
 'Utilization_CREDIT CARD_opened_last_36M',
'highest_sanction_amount_non_credit',
'Total_high_credit',
'num_PL_LE_1_Lac_opened_in_last_3M',
    
# banking
'cash_withdrawal_sum',
'balance_5th_day_M2',
'bounced_transactions_count',
'Min_Balance_Min_M1_M2_M3',
'num_credit_gt_salary',
'upi_trans_count',
'Max_Credit_Avg_M1_M2_M3',
'ratio_total_debit_to_total_credit_M3',
'ratio_6th_day_balance_to_2th_day_balance_M1',

# Reconcilated Features

'percentage_ever_delinquent',
'months_since_most_recent_delinquency_all',
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',

   

'g406s',
'dm206s',

#loan
'statedIncome',
'deviation_from_mean_income#last_30_Days',
'deviation_from_median_income/standard_deviation_income#last_365_Days',
'yearOfBirth_mean',
    
'month_year',
'G15_B24']

In [9]:

model_df = model_df1[features_list]
oot_df = oot_df1[features_list]

# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', 'G15_B24']) # Adjust 'target' to your actual target column name
y_model = model_df['G15_B24']

X_oot = oot_df.drop(columns=['month_year', 'G15_B24'])
y_oot = oot_df['G15_B24']

#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)



/tmp/ipykernel_3039734/2552150070.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)
/tmp/ipykernel_3039734/2552150070.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usi

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [11]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20606, 33), (13738, 33), (20606,), (13738,))

In [12]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(16484, 33) (4122, 33) (13738, 33)


In [13]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [14]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.3,
    'colsample_bytree': 0.3,
    'lambda': 6,
    'alpha': 4.8,
    'gamma': 5.0,
    'random_state': 24
}

evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 300,
    early_stopping_rounds= 40,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.50894	eval-auc:0.51238
[10]	train-auc:0.62167	eval-auc:0.59562
[20]	train-auc:0.62304	eval-auc:0.60549
[30]	train-auc:0.62351	eval-auc:0.60878
[40]	train-auc:0.62582	eval-auc:0.60602
[50]	train-auc:0.62785	eval-auc:0.60896
[60]	train-auc:0.62976	eval-auc:0.61441
[70]	train-auc:0.63171	eval-auc:0.61403
[80]	train-auc:0.63360	eval-auc:0.61203
[90]	train-auc:0.63348	eval-auc:0.61390
[100]	train-auc:0.63605	eval-auc:0.61661
[110]	train-auc:0.63610	eval-auc:0.61564
[120]	train-auc:0.63705	eval-auc:0.61465
[130]	train-auc:0.63877	eval-auc:0.61518
[140]	train-auc:0.63968	eval-auc:0.61737
[150]	train-auc:0.64028	eval-auc:0.61950
[160]	train-auc:0.64131	eval-auc:0.61972
[170]	train-auc:0.64189	eval-auc:0.62035
[180]	train-auc:0.64233	eval-auc:0.61966
[190]	train-auc:0.64299	eval-auc:0.62117
[200]	train-auc:0.64438	eval-auc:0.62160
[210]	train-auc:0.64454	eval-auc:0.62252
[220]	train-auc:0.64556	eval-auc:0.62411
[230]	train-auc:0.64548	eval-auc:0.62352
[240]	train-auc:0.64597	eva

In [15]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.65, Gini: 0.30, KS: 0.22
Eval AUC: 0.62, Gini: 0.25, KS: 0.20
Test AUC: 0.63, Gini: 0.25, KS: 0.19
OOT AUC: 0.63, Gini: 0.27, KS: 0.19


In [16]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.148575         0.111871        0.160140        0.099845   1649   
9          0.098908         0.090694        0.099844        0.083173   1648   
8          0.077670         0.078270        0.083166        0.073881   1648   
7          0.068526         0.070005        0.073866        0.066722   1649   
6          0.052791         0.064280        0.066722        0.062189   1648   
5          0.058252         0.060765        0.062187        0.059527   1648   
4          0.045482         0.058535        0.059526        0.057649   1649   
3          0.044296         0.056865        0.057648        0.056081   1648   
2          0.038228         0.055268        0.056080        0.054470   1648   
1          0.021831         0.053322        0.054470        0.051125   1649   

        label_good  label_bad  
Decile                         
10            1404        245  
9             1485        163  
8             1520        128  
7             1536        113  
6             1561         87  
5             1552         96  
4             1574         75  
3             1575         73  
2             1585         63  
1             1613         36

In [17]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.125910         0.111999        0.157311        0.100171   1374   
9          0.081514         0.090848        0.100132        0.083275   1374   
8          0.085881         0.078382        0.083272        0.073799   1374   
7          0.051712         0.070097        0.073793        0.066866   1373   
6          0.064047         0.064428        0.066865        0.062416   1374   
5          0.048763         0.060892        0.062416        0.059624   1374   
4          0.048070         0.058633        0.059623        0.057729   1373   
3          0.041485         0.056946        0.057728        0.056168   1374   
2          0.041485         0.055369        0.056167        0.054526   1374   
1          0.024017         0.053343        0.054524        0.051276   1374   

        label_good  label_bad  
Decile                         
10            1201        173  
9             1262        112  
8             1256        118  
7             1302         71  
6             1286         88  
5             1307         67  
4             1307         66  
3             1317         57  
2             1317         57  
1             1341         33

In [18]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.141536         0.112724        0.155266        0.100603   1081   
9          0.117593         0.091873        0.100595        0.084354   1080   
8          0.090741         0.079004        0.084354        0.074239   1080   
7          0.080556         0.070393        0.074237        0.066914   1080   
6          0.080556         0.064457        0.066912        0.062493   1080   
5          0.076852         0.061095        0.062492        0.059823   1080   
4          0.056481         0.058841        0.059821        0.057920   1080   
3          0.045370         0.057120        0.057920        0.056325   1080   
2          0.046296         0.055539        0.056321        0.054728   1080   
1          0.024977         0.053531        0.054726        0.050974   1081   

        label_good  label_bad  
Decile                         
10             928        153  
9              953        127  
8              982         98  
7              993         87  
6              993         87  
5              997         83  
4             1019         61  
3             1031         49  
2             1030         50  
1             1054         27

In [19]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
25                                              g406s        79.0
23                          Num_Enq_non_BANK_last_12M        65.0
13                         bounced_transactions_count        33.0
22                           No of enq in past 90days        24.0
16                                    upi_trans_count        19.0
7                          Age_of_oldest_open_account        18.0
12                                cash_withdrawal_sum        17.0
5                                     Utilization_all        16.0
0                      num_PERSONAL LOAN_enq_last_24M        16.0
1                       num_CONSUMER LOAN_enq_last_9M        16.0
15                               num_credit_gt_salary        16.0
6   Total_outstanding_Balance_CREDIT CARD_opened_l...        16.0
27                                       statedIncome        13.0
28            deviation_from_mean_income#last_30_Days        12.0
21        

In [20]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BBI_Train_Pred_1524.csv',index=False)
Eval.to_csv('LeoPayu_BBILA_Eval_Pred_1524.csv',index=False)
Test.to_csv('LeoPayu_BBILA_Test_Pred_1524.csv',index=False)
Oot.to_csv('LeoPayu_BBILA_Oot_Pred_1524.csv',index=False)


In [21]:
#Save the Model
xgb_model.save_model('LeoPayu_BBILA_15_in_24mob.json')
print('Model Saved')

Model Saved
